In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ast

from fastDamerauLevenshtein import damerauLevenshtein
from opportunistic_planning import data, sequence
from imp import reload
from scipy.stats import friedmanchisquare, wilcoxon
pd.set_option('display.max_rows', 500)

In [2]:
# read data

df = pd.read_csv('all_task_environments_2021-01-26.csv', header=0)

In [3]:
# dict generation ~1 sec

%time distances_dict = data.generate_distances_dict(df)

CPU times: user 1.12 s, sys: 85.2 ms, total: 1.21 s
Wall time: 1.11 s


In [6]:
# n=1: ~4min
# n=10: ~15min
# n=50: ~1h 44min
# n=100: 2h 4min
%time results_sum = data.calculate_prediction_error(df, distances_dict=distances_dict, error_function='prequential', dimensions=[[2, 'xy']], n=50)

CPU times: user 5min 56s, sys: 132 ms, total: 5min 56s
Wall time: 5min 55s


In [7]:
results_sum

,"c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.1; k: 0.0,0.1,1.1; xy","c: 1.2; k: 0.0,0.1,1.1; xy","c: 1.3; k: 0.0,0.1,1.1; xy","c: 1.4; k: 0.0,0.1,1.1; xy","c: 1.5; k: 0.0,0.1,1.1; xy","c: 1.6; k: 0.0,0.1,1.1; xy","c: 1.7; k: 0.0,0.1,1.1; xy","c: 1.8; k: 0.0,0.1,1.1; xy","c: 1.9; k: 0.0,0.1,1.1; xy",...,"c: 1.3; k: 0.8,0.9,1.9; xy","c: 1.4; k: 0.8,0.9,1.9; xy","c: 1.5; k: 0.8,0.9,1.9; xy","c: 1.6; k: 0.8,0.9,1.9; xy","c: 1.7; k: 0.8,0.9,1.9; xy","c: 1.8; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xy",sequence,error,ID
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,pocgkr,0.723,a1
1,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,cgwpcfks,0.785,a3
2,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,kfsfkspwg,0.806,a5
3,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,pfkswkfsococg,0.862,a11
4,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,wptgkfsoc,0.806,a13
5,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,pwfkg,0.666,a16
6,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,powcer,0.723,a19
7,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,coprerk,0.756,a22
8,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,wopofkkskf,0.824,a26
9,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,opwpwgcoc,0.806,a29


In [42]:
#results_new.T.to_csv('results/results_all_n50_2021-01-28.csv', header=True, index=True)

In [9]:
lowest_sum, lowest_idx_sum, lowest_median_sum, results_mean_sum = data.get_lowest_error(results_sum)
lowest_sum, lowest_idx_sum, lowest_median_sum

(4.10497557997558, Index(['c: 1.8; k: 0.2,0.3,1.3; xy'], dtype='object'), 4.0)

In [27]:
#results_sum.T.to_csv('results/results_2D_n50_2021-04-28.csv', header=True, index=True)

In [10]:
def get_list_sorted(results):
    
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean', col] = results[col].mean()
            results.loc['median', col] = results[col].median()
    lowest = sorted(results.loc['mean'])
    lowest_median = sorted(results.loc['median'])
    
    idx_list = []
                
    for i in lowest_median:
        idx = results.columns[(results.loc['median'] == i)]
        idx_list.append(idx)

    return lowest, results.columns[(results.loc['mean'] == lowest[0])], lowest_median, idx_list

In [13]:
cpt = [4.0, 6.0, 6.0, 10.0, 7.0, 3.0, 5.0, 6.0, 6.0, 8.0, 7.0, 5.0, 11.0, 4.0, 4.0, 5.0, 9.0, 5.0, 11.0, 12.0, 9.0, 6.0, 12.0, 14.0, 7.0, 6.0, 9.0, 8.0, 6.0, 3.0, 5.0, 10.0, 6.0, 9.0, 6.0, 8.0, 8.0, 6.0, 8.0, 4.0, 3.0, 3.0, 7.0, 4.0, 9.0, 2.0, 3.0, 7.0, 5.0, 5.0, 4.0, 3.0, 13.0, 5.0, 6.0, 9.0, 7.0, 6.0, 7.0, 4.0, 3.0, 11.0, 1.0, 5.0, 5.0, 3.0, 10.0, 12.0, 6.0, 4.0, 6.0, 2.0, 9.0, 7.0, 6.0, 10.0, 8.0, 5.0, 8.0, 6.0, 4.0, 4.0, 4.0, 4.0, 2.0, 4.0, 3.0, 2.0, 2.0, 2.0, 4.0, 5.0, 3.0, 5.0, 5.0, 8.0, 4.0, 1.0, 4.0, 4.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 3.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]

In [14]:
rnn = [2.0, 6.0, 6.0, 9.0, 6.0, 2.0, 2.0, 4.0, 7.0, 5.0, 7.0, 5.0, 11.0, 5.0, 3.0, 5.0, 7.0, 2.0, 10.0, 10.0, 7.0, 3.0, 11.0, 13.0, 5.0, 6.0, 6.0, 6.0, 4.0, 2.0, 5.0, 11.0, 6.0, 8.0, 3.0, 7.0, 9.0, 4.0, 7.0, 2.0, 5.0, 4.0, 6.0, 4.0, 6.0, 3.0, 3.0, 6.0, 3.0, 6.0, 2.0, 4.0, 12.0, 3.0, 3.0, 7.0, 6.0, 5.0, 6.0, 4.0, 1.0, 8.0, 1.0, 5.0, 4.0, 3.0, 7.0, 10.0, 4.0, 5.0, 5.0, 2.0, 9.0, 8.0, 4.0, 7.0, 9.0, 6.0, 4.0, 5.0, 2.0, 3.0, 2.0, 4.0, 2.0, 3.0, 3.0, 2.0, 2.0, 2.0, 4.0, 5.0, 3.0, 5.0, 5.0, 7.0, 3.0, 1.0, 3.0, 4.0, 4.0, 6.0, 6.0, 5.0, 4.0, 2.0, 6.0, 3.0, 5.0, 4.0, 6.0, 6.0, 1.0, 3.0, 3.0, 3.0, 6.0, 3.0, 5.0, 4.0, 5.0, 6.0, 4.0, 6.0, 6.0, 2.0, 6.0, 5.0, 6.0, 6.0, 5.0, 3.0, 6.0, 6.0, 6.0, 6.0, 4.0, 6.0, 5.0, 5.0, 3.0, 5.0, 5.0, 4.0, 5.0, 6.0, 4.0, 5.0, 5.0, 5.0, 5.0, 6.0, 2.0, 2.0, 3.0, 3.0, 3.0, 2.0, 5.0, 4.0, 1.0, 3.0, 2.0, 3.0, 3.0, 4.0, 6.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, 5.0, 3.0, 3.0, 3.0, 3.0, 3.0]

In [19]:
len(cpt), len(rnn)

(180, 180)

In [41]:
# get sequences where OPM performs worse than RNN

worse_than_rnn = results_sum[lowest_idx_sum[0]].loc[results_sum[lowest_idx_sum[0]] >= 5.0]
worse_than_rnn_sequences = [results_sum.at[x, 'sequence'] for x in worse_than_rnn.index]

worse_than_rnn_sequences

['cgwpcfks',
 'kfsfkspwg',
 'pfkswkfsococg',
 'wptgkfsoc',
 'coprerk',
 'wopofkkskf',
 'pocrfksfks',
 'pwocgfkfsrere',
 'pocpocwgkfs',
 'pwgockfsfksret',
 'ocpwgfkskfsrere',
 'copwgwpresk',
 'pocwggpfkfksrer',
 'wpofcgkrpwgfksocr',
 'cwpfksor',
 'pwfksorc',
 'pocgwkfkfser',
 'powctgcog',
 'powgcpo',
 'copcogpgwfksr',
 'wgptocopwg',
 'perkscwgtf',
 'pcgworkfkfs',
 'pwtgcfkfksr',
 'powcgkfkfr',
 'pocwcpoere',
 'pocfkswgr',
 'powcfkfk',
 'pwgfksfk',
 'cwopgtrekfksksf',
 'pwgterkfsoc',
 'pocgwrkfs',
 'tgcwpro',
 'pocgkfskfr',
 'wpgcowpcfksre',
 'pocrfkfk',
 'pockfskfswgre',
 'tgpcowpgpwcog',
 'gwpfrkst',
 'gwpocoskfsfk',
 'wtpcockfkf',
 'pococwtrfkfkg',
 'pwocfksfksr',
 'pwgerks',
 'pwfksocr',
 'mcbatp',
 'pcsbaoweg',
 'ocsfgkbp',
 'pofkscgb',
 'pfskbgco',
 'pfkscbgo',
 'pcgbfkos',
 'gcpbokfs',
 'sfkpcgbo',
 'spgokfcb',
 'cgokpbsf',
 'cogsfbpk',
 'opksfcgb',
 'pbgokfsc',
 'pkfoscbg',
 'pkosfgcb',
 'pkfoscgb',
 'pkfsgcbo',
 'pkfsocgb',
 'pokfscgb']

### Plot best model with baselines

In [16]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error = df['error']

seqs= results_sum['sequence'][:-2].values

res = results_sum[lowest_idx_sum[0]][:-2].values
median = [np.nanmedian(res)] * len(res)

x = [x for x in range (0,len(seqs))]

# plot CPT baseline
plt.scatter(x, cpt, marker='o', s=20, c='dodgerblue', alpha=0.3, 
            label=str('CPT baseline median: ') + str(round(np.median(cpt),3)))
plt.plot(x, [np.median(cpt)] * len(x), '-', c='dodgerblue', alpha=0.9, linewidth=2)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, cpt, alpha=0.3, color='dodgerblue')


# plot RNN baseline
plt.scatter(x, rnn, marker='o', s=20, c='limegreen', alpha=0.5, 
            label=str('RNN baseline median: ') + str(round(np.median(rnn),3)))
plt.plot(x, [np.median(rnn)] * len(x), '-', c='green', alpha=0.9, linewidth=2)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, rnn, alpha=0.3, color='limegreen')

# plot random baseline
#plt.scatter(x, error, c='red', s=20, marker='o', alpha=0.3, 
#            label=str('guessing baseline median: ') + str(round(np.nanmedian(results_new['error']),3)))
#plt.plot(x, [np.median(error)] * len(x), c='red', alpha=0.8)#, 
         #label=str('guessing baseline avg.: ' + str(round(np.mean(results_original['error']),2))))
#plt.fill_between(x, error, [1.0 * len(error)], color='red', alpha=0.2)

# plot scatter + lines for simulations
plt.scatter(x, res, marker='o', s=26, c='darkviolet', alpha=0.8, 
            label=str('model-generated median: ') + str(round(lowest_median_sum,3)))
#plt.plot(x, res, c='blue', alpha=0.6)
plt.plot(x, median, c='darkviolet', alpha=0.95, linewidth=2)


plt.xticks(x, labels=IDs, rotation=90, fontsize=5)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('accumulated prediction error', fontsize=22)
plt.xlabel('sequence', fontsize=22)
#plt.ylim(0.0, 0.51)
#plt.title('best model (all), ' + str(lowest_idx_sum[0]), fontsize=24, pad=20)
plt.margins(0.01)

plt.legend(fontsize=20, framealpha=0.8, loc='upper right', markerscale=2.5)

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

### Prep data for stat analysis

In [21]:
# Get errors for model
list_xy = []

for col in results_sum:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_sum.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))

Average xy: 4.379150104011215, stdev: 0.2094498245848324, median: 4.323214285714286


In [22]:
# model vs RNN
stat, p = wilcoxon(rnn, res, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 2916.500, p = 0.00002


In [23]:
# model vs CPT
stat, p = wilcoxon(cpt, res, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 96.000, p = 0.00000
